# Data Exploration using Spectogram visualizations

In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from scipy import signal

from data import Read_Data

In [2]:
gd = Read_Data.GestureData()
features = gd.compile_data(nfft=4096, overlap=0.5, brange=8, keras_format=False, plot_spectogram=True, baseline_format=False)

Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping files
Skipping f

In [3]:
from ipywidgets import interact
def make_plot(idx):
    plt.pcolormesh(features[idx]['features'])
    plt.title(str(features[idx]['user'])+' '+features[idx]['name'])
    plt.show()
    
interact(make_plot,idx=(0,len(features)-1,1),__manual=True)

<function __main__.make_plot>

In [4]:
len(features)

1419